# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
print(f'TensorFlow version: {tf.__version__}')
print(f'Keras version: {tf.keras.__version__}')

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 8
%config InlineBackend.figure_format = 'retina'

TensorFlow version: 2.3.1
Keras version: 2.4.0


In [2]:
print('GPU found :)' if tf.config.list_physical_devices("GPU") else 'No GPU :(')

No GPU :(


In [3]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.metrics import Recall

# Dataset informations

5 classes: 
- Bowl --> train = 1019 images / test = 565 images
- CanOfCocaCola --> train = 874 images / test = 671 images
- MilkBottle --> train = 1024 images / test = 642 images
- Rice --> train = 1032 images / test = 877 images
- Sugar --> train = 787 images / test = 813

# Data Loading

In [4]:
preprocess = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg19.preprocess_input
)

train_generator = preprocess.flow_from_directory(
        'DB/train',
        target_size=(227, 227),
        shuffle=True,
        class_mode='categorical')

test_generator = preprocess.flow_from_directory(
        'DB/test',
        target_size=(227, 227),
        shuffle=True,
        class_mode='categorical')

Found 4736 images belonging to 5 classes.
Found 3568 images belonging to 5 classes.


# Model creation and training

In [5]:
# Create our model from pretrained vgg16
vgg = VGG16(include_top=False, weights="imagenet")
vgg.trainable = False

dense = Sequential([
    Flatten(),
    Dense(512, activation="elu"),
    Dropout(0.1),
    Dense(256),
    Dense(5, activation="softmax")
], name="dense")

modele = Sequential([
    Input((227, 227, 3)),
    vgg,
    dense
], name="complet")

dense.summary()
modele.summary()

Model: "dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               12845568  
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1285      
Total params: 12,978,181
Trainable params: 12,978,181
Non-trainable params: 0
_________________________________________________________________
Model: "complet"
_________________________________________________________________
Layer (type)                 Out

In [7]:
# Compile, train and save model
recall = Recall(name="recall", thresholds=0.3)
modele.compile('adam', 'categorical_crossentropy', metrics=[recall])

history = modele.fit(train_generator, epochs=2,
                     validation_data=test_generator,
                     validation_steps=8,
                     verbose=1,
                     batch_size=32)

dense.save("modele-VGG16")

Epoch 1/2
148/148 [==============================] - 753s 5s/step - loss: 3.3749 - recall: 0.9582 - val_loss: 0.9812 - val_recall: 0.9297
Epoch 2/2
148/148 [==============================] - 727s 5s/step - loss: 0.0424 - recall: 0.9949 - val_loss: 1.2581 - val_recall: 0.9375


In [ ]:
# Evaluate model performance on test data
_, test_acc = modele.evaluate(test_generator, verbose=0)

print(f"Test accuracy: {test_acc:.5f}")